In [1]:
# Install necessary packages
%pip install -qU pypdf==4.0.1 langchain_community
%pip install -U duckduckgo-search
%pip install -qU langchain-openai
%pip install faiss-cpu
# %pip install faiss-gpu

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install langchain==0.3.3

Note: you may need to restart the kernel to use updated packages.


In [3]:
import os
from langchain_community.document_loaders import PyPDFLoader
from langchain.agents import initialize_agent, AgentType
from langchain.tools import DuckDuckGoSearchResults, StructuredTool
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.vectorstores import FAISS  # Corrected import
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA
from langchain.tools import Tool
from pydantic import BaseModel, Field
from langchain.chains.router import MultiRetrievalQAChain

In [4]:
import langchain
print(langchain.__version__)

0.3.3


In [5]:
import os
api_key = ""
os.environ["OPENAI_API_KEY"] = api_key
os.environ["API_KEY_OPENAI"] = api_key

In [6]:
# List of company names and their corresponding PDF file paths
companies = {
    "Nike": "../../data/Nike-Annual-Report.pdf",
    "Lyft": "../../data/Lyft-Annual-Report-2021.pdf",
    "Uber": "../../data/Uber-Annual-Report-2021.pdf"
}

In [7]:
# List of company names and their corresponding PDF file paths
companies = {
    "Nike": "../../data/Nike-Annual-Report.pdf",
    "Lyft": "../../data/Lyft-Annual-Report-2021.pdf",
    "Uber": "../../data/Uber-Annual-Report-2021.pdf"
}


In [8]:

# Initialize dictionaries to hold data for each company
company_docs = {}
company_vectorstores = {}
company_retrievers = {}
company_qa_tools = {}


In [9]:

# Initialize the text splitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)

# Process each company's PDF
for company, file_path in companies.items():
    # Load the PDF document
    loader = PyPDFLoader(file_path)
    docs = loader.load()
    
    # Split the document into chunks
    splits = text_splitter.split_documents(docs)
    
    # Create embeddings
    embeddings = OpenAIEmbeddings()
    
    # Create a vector store for the company's documents using FAISS
    vectorstore = FAISS.from_documents(documents=splits, embedding=embeddings)
    
    # Create a retriever for the vector store
    retriever = vectorstore.as_retriever()
    
    # Store in dictionaries
    company_docs[company] = docs
    company_vectorstores[company] = vectorstore
    company_retrievers[company] = retriever

llm = ChatOpenAI(model="gpt-4")  # Ensure you have access to GPT-4


/opt/anaconda3/envs/llama_index_recursive/lib/python3.11/site-packages/pypdf/_crypt_providers/_cryptography.py:32: CryptographyDeprecationWarning: ARC4 has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.ARC4 and will be removed from this module in 48.0.0.
  from cryptography.hazmat.primitives.ciphers.algorithms import AES, ARC4
/var/folders/lm/w5nd7ynn3n76b25r6j68r0g40000gn/T/ipykernel_48108/637060213.py:14: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings()
/var/folders/lm/w5nd7ynn3n76b25r6j68r0g40000gn/T/ipykernel_48108/637060213.py:27: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be 

In [10]:

# For each company, create a RetrievalQA chain and wrap it into a Tool
for company, retriever in company_retrievers.items():
    # Create the RetrievalQA chain
    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",  # You can choose 'stuff', 'map_reduce', etc.
        retriever=retriever
    )
    
    # Wrap the chain into a Tool
    qa_tool = Tool(
        name=f"{company} QA",
        func=qa_chain.run,
        description=(
            f"Use this tool to answer questions about {company}'s financial information."
        )
    )
    
    # Store the tool
    company_qa_tools[company] = qa_tool

# # Create a dictionary mapping company names to their retrievers
# retriever_dict = {
#     company: retriever for company, retriever in company_retrievers.items()
# }

# # Optionally, define retriever prompts
# retriever_prompts = {
#     company: f"Good for answering questions about {company}." for company in companies.keys()
# }


In [11]:
retriever_infos = [
    {
        "name": "Nike_Retriever",
        "description": "Retriever for Nike related queries and documents",
        "retriever": company_retrievers['Nike'],  # Instance of your first retriever
    },
    {
        "name": "Lyft_Retriever",
        "description": "Retriever for Lyft related queries and documents",
        "retriever": company_retrievers['Lyft'],  # Instance of your second retriever
    },
    {
        "name": "Uber_Retriever",
        "description": "Retriever for Uber related queries and documents",
        "retriever": company_retrievers['Uber'],  # Instance of your third retriever
    }
]

In [12]:

# Create the MultiRetrievalQAChain
multi_retrieval_chain = MultiRetrievalQAChain.from_retrievers(
    llm=llm,
    retriever_infos = retriever_infos,
    default_chain=None,
    verbose=True,
    default_chain_llm=llm
)

# Create a Tool that wraps the multi_retrieval_chain
multi_qa_tool = Tool(
    name="Company QA",
    func=multi_retrieval_chain.run,
    description="Use this tool to answer questions about Nike, Lyft, or Uber. Especially related to their financials"
)


/opt/anaconda3/envs/llama_index_recursive/lib/python3.11/site-packages/pydantic/main.py:212: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  validated_self = self.__pydantic_validator__.validate_python(data, self_instance=self)


In [13]:

# Define the revenue growth calculation tool with input schema
class RevenueGrowthInput(BaseModel):
    revenue_new: float = Field(..., description="Revenue in the new year")
    revenue_old: float = Field(..., description="Revenue in the old year")

def calculate_revenue_growth(revenue_new: float, revenue_old: float) -> float:
    """Calculates revenue growth from the old year to the new year."""
    return ((revenue_new - revenue_old) / revenue_old) * 100

calculate_growth_tool = StructuredTool.from_function(
    func=calculate_revenue_growth,
    name="Revenue Growth Calculator",
    description=(
        "Calculates the revenue growth percentage between two years. "
        "Requires 'revenue_new' and 'revenue_old' as inputs."
    ),
    args_schema=RevenueGrowthInput
)

# Define the DuckDuckGo search tool
web_search_tool = DuckDuckGoSearchResults(
    name="Web Search",
    description=(
        "Use this tool to search the web for financial information such as revenue figures "
        "if they are not found in the documents."
    )
)

# Update the tools list
tools = [calculate_growth_tool, web_search_tool, multi_qa_tool]

# Define the system message
system_message = """
You are an assistant that helps answer financial questions about companies.

When given a question, you should:

1. Determine which company's information is relevant to the question.

2. Use the 'Company QA' tool to find information from the appropriate company's documents.

3. If information is missing, use the 'Web Search' tool to find it.

4. Once you have all the necessary information, if you need to do some calculation regartding revenue growth then use the 'Revenue Growth Calculator' tool to compute or provide the answer.

Use the tools as needed, and be sure to provide the final answer to the user.
"""

agent_kwargs = {
    "system_message": system_message
}

# Initialize the agent
agent_chain = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    agent_kwargs=agent_kwargs
)

# Test queries
queries = [
    "What is the revenue growth between 2022 and 2023 for Lyft?",
    "Summarize Uber's financial performance in 2022.",
    "What are Nike's net profits for the last quarter?"
]

for query in queries:
    print(f"\nQuery: {query}")
    results = agent_chain.run(query)
    print(f"Answer: {results}")


/var/folders/lm/w5nd7ynn3n76b25r6j68r0g40000gn/T/ipykernel_48108/2448552257.py:54: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 1.0. Use :meth:`~Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc.` instead.
  agent_chain = initialize_agent(
/var/folders/lm/w5nd7ynn3n76b25r6j68r0g40000gn/T/ipykernel_48108/2448552257.py:71: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  results = agent_chain.run(query)



Query: What is the revenue growth between 2022 and 2023 for Lyft?


> Entering new AgentExecutor chain...
Thought: I don't have the specific revenue figures for Lyft for the years 2022 and 2023. I need to find this information first. I will use the Web Search tool to find out the revenue of Lyft for these years.

Action:
```
{
  "action": "Web Search",
  "action_input": {
    "query": "Lyft revenue 2022"
  }
}
```
Observation: snippet: Lyft Media: Launched in-app video ads in Q4 with strong results in terms of views and click-throughs. Lyft Media revenue in Q4 2023 surpassed the level achieved in all of 2022. We are working closely with partners to create great experiences for customers, tapping into our lifestyle and destination targeting capabilities., title: Lyft Announces Fourth Quarter and Full-Year 2023 Results, link: https://investor.lyft.com/news-and-events/news/news-details/2024/Lyft-Announces-Fourth-Quarter-and-Full-Year-2023-Results/default.aspx, snippet: Lyft Key Statistic

In [14]:
results = agent_chain.run("Summarize the lyft document. What info does it mention?")
print(f"Answer: {results}")



> Entering new AgentExecutor chain...
Action:
{
  "action": "Company QA",
  "action_input": "Summarize the lyft document"
}

> Finished chain.
Answer: Action:
{
  "action": "Company QA",
  "action_input": "Summarize the lyft document"
}


In [15]:
results = agent_chain.run("What is Nike's net profit after the end of May 31, 2023?")
print(f"Answer: {results}")



> Entering new AgentExecutor chain...
Thought: I don't have this information readily available. I will need to use the Company QA tool to answer the question about Nike's financials.
Action:
```
{
  "action": "Company QA",
  "action_input": "What is Nike's net profit after the end of May 31, 2023?"
}
```

> Entering new MultiRetrievalQAChain chain...
Nike_Retriever: {'query': "What is Nike's net profit after the end of May 31, 2023?"}
> Finished chain.

Observation: Nike's net income after the end of May 31, 2023 was $5,070 million.
Thought:Now that I have the information, I can provide the answer to the user. 
Action:
```
{
  "action": "Final Answer",
  "action_input": "Nike's net income after the end of May 31, 2023 was $5,070 million."
}
```

> Finished chain.
Answer: Nike's net income after the end of May 31, 2023 was $5,070 million.
